In [349]:
conn.close()

In [120]:
import psycopg2
import pandas as pd
import sqlite3

In [121]:
### Connect to ElephantSQL-hosted PostgreSQL
conn = psycopg2.connect(dbname='waizzkeg', user='waizzkeg',
                        password='qn83fIuPrDvRhAS7Th4banRwREmnA3xm', host='otto.db.elephantsql.com')
### A "cursor", a structure to iterate over db records to perform queries
cur = conn.cursor()


In [122]:
### An example query
cur.execute('SELECT * from test_table;')
### Note - nothing happened yet! We need to actually *fetch* from the cursor
cur.fetchone()

(1, 'A row name', None)

In [123]:
df = pd.read_csv('titanic.csv')

df.info

<bound method DataFrame.info of      Survived  Pclass  \
0           0       3   
1           1       1   
2           1       3   
3           1       1   
4           0       3   
5           0       3   
6           0       1   
7           0       3   
8           1       3   
9           1       2   
10          1       3   
11          1       1   
12          0       3   
13          0       3   
14          0       3   
15          1       2   
16          0       3   
17          1       2   
18          0       3   
19          1       3   
20          0       2   
21          1       2   
22          1       3   
23          1       1   
24          0       3   
25          1       3   
26          0       3   
27          0       1   
28          1       3   
29          0       3   
..        ...     ...   
857         0       2   
858         1       1   
859         0       3   
860         0       2   
861         1       2   
862         1       2   
863         0     

In [124]:
pd.options.display.max_colwidth = 100
df.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cumings,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [125]:
df['Sex'] = df['Sex'].replace({'female':0,'male':1})


In [126]:
str(tuple(i for i in list(df['Pclass'].value_counts().index)))

'(3, 1, 2)'

In [127]:
cur.execute('''CREATE TYPE pclass AS ENUM '''+str(tuple(str(i) for i in list(df['Pclass'].value_counts().index))))


In [128]:
print(cur.execute('''SELECT enum_range(NULL::pclass)''').fetchall())

AttributeError: 'NoneType' object has no attribute 'fetchall'

In [357]:
#conn = psycopg2.connect(dbname='waizzkeg', user='waizzkeg',
                       # password='qn83fIuPrDvRhAS7Th4banRwREmnA3xm', host='otto.db.elephantsql.com')
def create_db(csv, dbname, user, password, host):
    
    conn = psycopg2.connect(dbname=dbname, user=user,
                        password=password, host=host)
### A "cursor", a structure to iterate over db records to perform queries
    cur = conn.cursor()
    df = pd.read_csv(csv)
    def enum_pclass(file):
        with open(file, "r", encoding='utf-8') as f:
            reader = csv.reader(f)
            i = next(reader)
            lines = f.readlines()
            enums = []
            for line in lines:
                enum = line.split(',',2)[1]
                enums.append(enum)
            t_enums = tuple(set(enums))
        return t_enums
    cur.execute('''DROP TABLE titanic''')
    cur.execute('''DROP TYPE pclass''')
    cur.execute('''CREATE TYPE pclass AS ENUM '''+str(t_enums))
    
    create_table_query_type = '''CREATE TABLE titanic (id SERIAL PRIMARY KEY, survived BOOL NOT NULL,
    pclass pclass, name VARCHAR(255) NOT NULL, sex BOOL NOT NULL, age FLOAT, s_s INT, p_c INT, fare FLOAT);'''
    
    cur.execute(create_table_query_type)
    conn.commit()
    return conn


def create_table(file, conn):
    cur = conn.cursor()
    with open(file, "r", encoding='utf-8') as f:
            reader = csv.reader(f)
            i = next(reader)
            lines = f.readlines()
            for line in lines:
                    survived, pclass = line.split(',',2)[:2]
                    name = line.split(',',3)[2]
                    if line.split(',',4)[3] == 'male':
                        sex =  line.split(',',4)[3].replace('male', '0')
                    else:
                        sex = line.split(',',4)[3].replace('female', '1')
                    age, s_s, p_c = line.split(',')[4:-1] 
                    fare = line.split(',')[-1].replace('\n','')
                    cur.execute('''INSERT INTO titanic (survived, pclass, name, sex, age, s_s, p_c, fare)
                        VALUES(%s, %s, %s, %s, %s, %s,%s, %s);''', (survived, str(pclass), str(name), sex, age, s_s, p_c, fare)
                    )
    conn.commit()
    cur.execute('''SELECT * FROM titanic''')
    print(f'Return from query: {cur.fetchall()}\nSuccess!!!')
                

In [358]:
conn.close()
create_table('titanic.csv', create_db('titanic.csv', dbname='waizzkeg', user='waizzkeg',
                        password='qn83fIuPrDvRhAS7Th4banRwREmnA3xm', host='otto.db.elephantsql.com'))

Return from query: [(1, False, '3', 'Mr. Owen Harris Braund', False, 22.0, 1, 0, 7.25), (2, True, '1', 'Mrs. John Bradley (Florence Briggs Thayer) Cumings', True, 38.0, 1, 0, 71.2833), (3, True, '3', 'Miss. Laina Heikkinen', True, 26.0, 0, 0, 7.925), (4, True, '1', 'Mrs. Jacques Heath (Lily May Peel) Futrelle', True, 35.0, 1, 0, 53.1), (5, False, '3', 'Mr. William Henry Allen', False, 35.0, 0, 0, 8.05), (6, False, '3', 'Mr. James Moran', False, 27.0, 0, 0, 8.4583), (7, False, '1', 'Mr. Timothy J McCarthy', False, 54.0, 0, 0, 51.8625), (8, False, '3', 'Master. Gosta Leonard Palsson', False, 2.0, 3, 1, 21.075), (9, True, '3', 'Mrs. Oscar W (Elisabeth Vilhelmina Berg) Johnson', True, 27.0, 0, 2, 11.1333), (10, True, '2', 'Mrs. Nicholas (Adele Achem) Nasser', True, 14.0, 1, 0, 30.0708), (11, True, '3', 'Miss. Marguerite Rut Sandstrom', True, 4.0, 1, 1, 16.7), (12, True, '1', 'Miss. Elizabeth Bonnell', True, 58.0, 0, 0, 26.55), (13, False, '3', 'Mr. William Henry Saundercock', False, 20.0, 

In [129]:
import csv

In [137]:
reader = csv.reader('titanic.csv')
for i in next(reader):
    print(i)

t


In [321]:
import csv
with open("titanic.csv", "r", encoding='utf-8') as f:
    reader = csv.reader(f)
    i = next(reader)
    survived, pclass, name, sex, age, s_s, p_c = str(line).split(',')[:-1]
    print(survived, pclass, str(name), sex, age, s_s, p_c)

0 3 Mr. Patrick Dooley male 32 0 0


In [ ]:
import csv
def enum_pclass(file):
    with open(file, "r", encoding='utf-8') as f:
        reader = csv.reader(f)
        i = next(reader)
        lines = f.readlines()
        enums = []
        for line in lines:
            survived, pclass, name, sex, age, s_s, p_c = line.split(',')[:-1]
            fare = line.split(',')[-1].replace('\n','')
            query = '''INSERT INTO titanic (survived, pclass, name, sex, age, s_s, p_c, fare)
                VALUES({}, {},{}, {},{}, {},{}, {})'''.format(survived, pclass, name, sex, age, s_s, p_c, fare)
            cur.execute(query)
    conn.cursor()
    cur.execute('''SELECT * FROM titanic''')
    print(f'Return from query: {curr.fetchall()}\nSuccess!!!')
    return t_enums


In [232]:
enum_pclass("titanic.csv")

('3', '1', '2')

In [198]:
tuple??

In [143]:
next(reader)

ValueError: I/O operation on closed file.

In [144]:
lines = f.readlines()

ValueError: I/O operation on closed file.